In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms.base import LLM


In [60]:
load_dotenv()

True

In [61]:
source = pd.read_parquet('../../data/processed/cleanedData.parquet')

In [62]:
class openRouterLLM(LLM):
    model: str
    key: str

    def _call(self, prompt, stop=None, run_manager=None):
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.key,
        )
        
        completion = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        return completion.choices[0].message.content
    
    @property
    def _llm_type(self):
        return "openrouter"    

In [63]:
llm = openRouterLLM(
    model=os.getenv('MODEL2'),
    key=os.getenv('KEY')
)

In [64]:
agentExecutor = create_pandas_dataframe_agent(llm, source, verbose=False, handle_parsing_errors=True, allow_dangerous_code=True)

d:\academic\Spark\RetailRadar\.venv\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [65]:
def askQuestion(quiz):
    response = agentExecutor.run(quiz)
    return response

In [66]:
print(askQuestion("What is the average age?"))

TypeError: 'NoneType' object is not subscriptable